In [9]:
import pandas as pd

df = pd.read_csv('../data/game_data.csv')

df = df.drop(columns=['scrape_id', 'main_story_polled', 'main_+_extras_polled', 'completionist_polled', 'all_playstyles_polled', 'developers', 'publishers', 'aliases', 'alias'])

df = df.dropna(subset=['genres'])
df = df.dropna(subset=['platforms'])

df['rating'] = pd.to_numeric(df['rating'], errors="coerce")
df = df.dropna(subset=['rating'])
df['rating'] = df['rating'] * 100
df['rating'] = df['rating'].astype(int)


df = df.dropna(subset=['na', 'eu', 'jp'], how='all')
df['eu'] = df['eu'].fillna(df['na'])
df['jp'] = df['jp'].fillna(df['na'])
df['eu'] = df['eu'].fillna(df['jp'])
df['jp'] = df['jp'].fillna(df['eu'])
df['na'] = df['na'].fillna(df['eu'])
df['eu'] = pd.to_datetime(df['eu'])
df['na'] = pd.to_datetime(df['na'])
df['jp'] = pd.to_datetime(df['jp'])

df = df.dropna(subset=['developer', 'publisher'], how='all')
df['developer'].fillna(df['publisher'], inplace=True)
df['publisher'].fillna(df['developer'], inplace=True)

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        if "median" not in col:
            df = df.drop(columns=[col])
        else:
            df[col] = pd.to_timedelta(df[col], errors="coerce")

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        df[col].fillna(df[col].mean(), inplace=True)

missing_values = df.isnull().sum()
print(missing_values)



name                     0
rating                   0
beat_count               0
platforms                0
genres                   0
developer                0
publisher                0
na                       0
main_story_median        0
main_+_extras_median     0
completionist_median     0
all_playstyles_median    0
eu                       0
jp                       0
dtype: int64


In [8]:
df['developer_count'] = df.groupby('developer')['name'].transform('count')
df['publisher_count'] = df.groupby('publisher')['name'].transform('count')

from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# X = np.array(df['developer_count']).reshape(-1, 1)
# wcss = []
# for i in range(1, 11):
#     kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()
#

kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
df['developer_size'] = kmeans.fit_predict(X)

X = np.array(df['publisher_count']).reshape(-1, 1)
# wcss = []
# for i in range(1, 11):
#     kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
df['publisher_size'] = kmeans.fit_predict(X)


avg_game_treshold = 70
def assign_group(value):
    if value <= avg_game_treshold:
        return 0
    else:
        return 1
    # else:
    #     return 2

df['rating_group'] = df['rating'].apply(assign_group)

import matplotlib.pyplot as plt

# Vytvoření sloupce s hodnoceními
rating_counts = df['rating_group'].value_counts()

# Vytvoření grafu
plt.figure(figsize=(8, 6))
plt.bar(rating_counts.index, rating_counts.values)
plt.xlabel('Hodnocení')
plt.ylabel('Počet her')
plt.title('Zastoupení hodnocení her')
plt.xticks(rating_counts.index)
plt.show()

df = df.drop(columns=["developer","rating", "publisher", "name", "developer_count", "publisher_count", "all_playstyles_median"])



NameError: name 'X' is not defined

In [ ]:
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler

df["genres"] = df["genres"].apply(literal_eval)
df["platforms"] = df["platforms"].apply(literal_eval)

df['is_PC'] = df['platforms'].apply(lambda x: 1 if 'PC' in x else 0)
df['is_Console'] = df['platforms'].apply(lambda x: 1 if 'PC' not in x else 0)


# Rozbití listů na samostatné řádky
exploded_genres = df['genres'].explode()

# Výpočet četností jednotlivých žánrů
genre_counts = exploded_genres.value_counts()
rare_genres = genre_counts[genre_counts < 50].index

mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df['genres']), columns=mlb.classes_, index=df.index)

data_encoded = pd.concat([df , genres_encoded], axis=1)

data_encoded['na_year'] = data_encoded['na'].dt.year
data_encoded['na_month'] = data_encoded['na'].dt.month
data_encoded['na_day'] = data_encoded['na'].dt.day

data_encoded = data_encoded.drop(columns=["genres", "platforms", "na", "jp", "eu"])

for col in data_encoded.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        data_encoded[col] = data_encoded[col].dt.total_seconds() / 60

columns_to_check = ['main_story_median', 'main_+_extras_median', 'completionist_median']
threshold = 3  #

filtered_data = data_encoded.copy()

for column in columns_to_check:
    z_scores = np.abs((data_encoded[column] - data_encoded[column].mean()) / data_encoded[column].std())
    filtered_data = filtered_data[z_scores < threshold]

data_encoded = filtered_data

scaler = MinMaxScaler()

data_encoded['main_story_normalized'] = scaler.fit_transform(data_encoded[['main_story_median']])
data_encoded['main_+_extras_normalized'] = scaler.fit_transform(data_encoded[['main_+_extras_median']])
data_encoded['completionist_normalized'] = scaler.fit_transform(data_encoded[['completionist_median']])
data_encoded['beat_count'] = scaler.fit_transform(data_encoded[['beat_count']])

data_encoded = data_encoded.drop(columns=[*columns_to_check, *rare_genres])

data_encoded

/var/folders/cx/lhmwf8r906d7f7785t7lwcdh0000gn/T/ipykernel_27559/271219847.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[z_scores < threshold]
/var/folders/cx/lhmwf8r906d7f7785t7lwcdh0000gn/T/ipykernel_27559/271219847.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[z_scores < threshold]


,beat_count,developer_size,publisher_size,rating_group,is_PC,is_Console,Action,Adventure,Arcade,Beat em Up,...,Tower Defense,Turn-Based,Virtual Reality,Visual Novel,na_year,na_month,na_day,main_story_normalized,main_+_extras_normalized,completionist_normalized
0,0.004609,0,0,1,0,1,1,0,0,0,...,0,0,0,0,2009,7,22,0.078904,0.041655,0.030806
1,0.007367,0,0,1,1,0,1,0,0,0,...,0,0,0,0,2006,10,24,0.186035,0.100070,0.069134
2,0.003967,0,0,1,1,0,1,0,0,0,...,0,0,0,0,2007,5,8,0.199901,0.125104,0.088798
3,0.002871,0,0,1,1,0,1,0,0,0,...,0,0,0,0,2008,3,25,0.207659,0.132684,0.093730
4,0.010502,0,0,1,0,1,1,0,0,0,...,0,0,0,0,2003,2,11,0.168868,0.083380,0.056150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,0.007669,0,0,0,1,0,0,1,0,0,...,0,0,0,0,2022,11,18,0.069000,0.035675,0.023534
10777,0.002758,0,1,1,1,0,1,0,0,0,...,0,0,0,0,2022,10,27,0.311654,0.191864,0.131059
10778,0.000189,2,1,1,1,0,0,0,0,0,...,0,0,0,0,2021,9,24,0.011225,0.006885,0.002778
10779,0.000491,0,0,1,1,0,0,0,0,0,...,0,0,0,0,2021,11,1,0.019478,0.008623,0.004651


In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, roc_auc_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

duplicate_column_names = data_encoded.columns[data_encoded.columns.duplicated()]
data_encoded = data_encoded.drop(columns=duplicate_column_names)

class_distribution = data_encoded['rating_group'].value_counts()
print(class_distribution)

X = data_encoded.drop('rating_group', axis=1)
y = data_encoded['rating_group']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier(max_depth=30,n_estimators=120,random_state=1)

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test, y_pred))


f_score = f1_score(y_test, y_pred, average='macro')

print("F-score: ", f_score)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)

if len(set(y_test)) == 2:
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC Score: ", roc_auc)
else:
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_lb = lb.transform(y_test)
    y_pred_lb = lb.transform(y_pred)
    roc_auc = roc_auc_score(y_test_lb, y_pred_lb, average='macro')
    print("ROC AUC Score: ", roc_auc)

y_pred_prob = model.predict_proba(X_test)
logloss = log_loss(y_test, y_pred_prob)
print("Log Loss: ", logloss)

rating_group
0    4936
1    4905
Name: count, dtype: int64
Accuracy:  0.7404773996952768
F-score:  0.7403356779432467
Confusion Matrix: 
 [[752 238]
 [273 706]]
ROC AUC Score:  0.7403699920553853
Log Loss:  0.5302427777968443


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [25, 30, 35],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_model = XGBClassifier(eval_metric='mlogloss')

grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='accuracy')
grid_search_xgb.fit(X_train, y_train)

print("Nejlepší hyperparametry pro XGBClassifier: ", grid_search_xgb.best_params_)

best_xgb = grid_search_xgb.best_estimator_

y_pred_xgb = best_xgb.predict(X_test)

print("XGBClassifier Accuracy: ", accuracy_score(y_test, y_pred_xgb))

f_score_xgb = f1_score(y_test, y_pred_xgb, average='macro')
print("XGBClassifier F-score: ", f_score_xgb)

cm = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix: \n", cm)

if len(set(y_test)) == 2:
    roc_auc = roc_auc_score(y_test, y_pred_xgb)
    print("ROC AUC Score: ", roc_auc)
else:
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_lb = lb.transform(y_test)
    y_pred_lb = lb.transform(y_pred_xgb)
    roc_auc = roc_auc_score(y_test_lb, y_pred_lb, average='macro')
    print("ROC AUC Score: ", roc_auc)

y_pred_prob = best_xgb.predict_proba(X_test)
logloss = log_loss(y_test, y_pred_prob)
print("Log Loss: ", logloss)


Nejlepší hyperparametry pro XGBClassifier:  {'learning_rate': 0.1, 'max_depth': 30, 'n_estimators': 100}
XGBClassifier Accuracy:  0.7140680548501778
XGBClassifier F-score:  0.7136852148251307
Confusion Matrix: 
 [[739 251]
 [312 667]]
ROC AUC Score:  0.7138860515265011
Log Loss:  0.5997990022997985


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

param_grid_svc = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': [1, 0.1, 0.01]
}

svc_model = make_pipeline(StandardScaler(), SVC(probability=True))

grid_search_svc = GridSearchCV(svc_model, param_grid_svc, cv=3, scoring='accuracy')
grid_search_svc.fit(X_train, y_train)

print("Nejlepší hyperparametry pro SVC: ", grid_search_svc.best_params_)

best_svc = grid_search_svc.best_estimator_

y_pred_svc = best_svc.predict(X_test)

print("SVC Accuracy: ", accuracy_score(y_test, y_pred_svc))

f_score_svc = f1_score(y_test, y_pred_svc, average='macro')
print("SVC F-score: ", f_score_svc)

cm = confusion_matrix(y_test, y_pred_svc)
print("Confusion Matrix: \n", cm)

if len(set(y_test)) == 2:
    roc_auc = roc_auc_score(y_test, y_pred_svc)
    print("ROC AUC Score: ", roc_auc)
else:
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_lb = lb.transform(y_test)
    y_pred_lb = lb.transform(y_pred_svc)
    roc_auc = roc_auc_score(y_test_lb, y_pred_lb, average='macro')
    print("ROC AUC Score: ", roc_auc)

y_pred_prob = best_svc.predict_proba(X_test)
logloss = log_loss(y_test, y_pred_prob)
print("Log Loss: ", logloss)


Nejlepší hyperparametry pro SVC:  {'svc__C': 10, 'svc__gamma': 0.01}
SVC Accuracy:  0.6988318943626206
SVC F-score:  0.6962938031156223
Confusion Matrix: 
 [[778 212]
 [381 598]]
ROC AUC Score:  0.6983429803654522
Log Loss:  0.573849630741482
